# Dawid-Skene Algorithm using the diagonal of the confucion matrix

if we use the diagonal of the confusion matrix $\pi$ we assume:
- annotators only have a probabilty of labeling the correct class correctly


In [3]:
# import super small toy dataset
from simple_toy_data import votes as svotes, N_WORKERS as sN_WORKERS, N_CLASSES as sN_CLASSES


```python
# those are equivalent
pij = np.einsum("j,jil->il", dsd.T[:,1], dsd.crowd_matrix) 
pij = dsd.T[:, 1] @ dsd.crowd_matrix.transpose((1, 0, 2))
# then denomiator:
denom = np.einsum("j,jil->i", dsd.T[:,1], dsd.crowd_matrix) 
```

In [4]:
import warnings
from os import PathLike
from sys import getsizeof
from typing import Annotated, Generator

import numpy as np
import sparse as sp
from annotated_types import Ge
from loguru import logger
from memory_profiler import profile
from pydantic import validate_call
from tqdm.auto import tqdm

from peerannot.models.template import CrowdModel,AnswersDict
from peerannot.models import DawidSkene
# load data
from toy_data import votes, N_CLASSES, N_WORKERS

from types import MethodType

/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


class DawidSkeneDiagonal(DawidSkene):
    """
    =============================
    Dawid and Skene model (1979)
    =============================

    Assumptions:
    - independent workers

    Using:
    - EM algorithm

    Estimating:
    - Different diagonal matrix for each worker
    """

    def _m_step(self) -> None:
        """Maximizing log likelihood with only diagonal elements of pi."""
        rho = self.T.sum(0) / self.n_task

        pi = np.zeros((self.n_workers, self.n_classes))
        for j in range(self.n_classes):
            # TODO:@jzftran change this to calculate diagonal faster
            pij = self.T[:, j] @ self.crowd_matrix.transpose((1, 0, 2))
            denom = pij.sum(1)


            diag_values = pij[:, j] / np.where(denom > 0, denom, 1e-9)
            pi[:, j] = diag_values


        # pi shape (n_workers, n_class), reresents how sure worker is sure that the label j is true
        pi_non_diag_values = (np.ones_like(pi) - pi) / (self.n_classes - 1)
        self.rho, self.pi, self.pi_non_diag_values = rho, pi, pi_non_diag_values


    def _e_step(self) -> None:
        """Estimate indicator variables (see eq. 2.5 Dawid and Skene 1979)

        Returns:
            T: New estimate for indicator variables (n_task, n_worker)
            denom: value used to compute likelihood easily
        """
        T = np.zeros((self.n_task, self.n_classes))

        worker_probs = np.zeros(self.n_workers)
        
        for i in range(self.n_task):
            for j in range(self.n_classes):
                # Compute per-worker probabilities
                for k in range(self.n_workers):
                    # Worker k's annotations for task i
                    worker_labels = self.crowd_matrix[i, k, :]  # shape (n_classes,)
                    
                    # Extract the diagonal and off-diagonal contributions
                    diag_contrib = np.power( self.pi[k, j], worker_labels[j])  # When label j is actually assigned
                    off_diag_contrib = np.prod(
                        np.power(self.pi_non_diag_values[k, j] , worker_labels[np.arange(self.n_classes) != j])
                    )

                    worker_probs[k] = diag_contrib * off_diag_contrib

                num = np.prod(worker_probs) * self.rho[j]
                T[i, j] = num

        self.denom_e_step = T.sum(1, keepdims=True)
        T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)
        self.T = T


In [13]:
dsd = DawidSkeneDiagonal(
    answers=votes, n_workers=N_WORKERS, n_classes=N_CLASSES, sparse=False
)
dsd.run(maxiter=100)
dsd.get_answers()

2025-03-25 10:17:05.547 | DEBUG    | __main__:init_crowd_matrix:95 - Dense crowd matrix  5904
2025-03-25 10:17:05.549 | DEBUG    | __main__:init_T:103 - Size of T before calc: 1568
2025-03-25 10:17:05.550 | DEBUG    | __main__:init_T:107 - Size of T: 1568
Finished:  59%|█████▉    | 59/100 [00:00<00:00, 112.91it/s]       


array([2, 2, 8, 1, 4, 0, 5, 3, 7, 1, 0, 3, 1, 3, 4, 7, 4, 7, 3, 7])

In [10]:
# The fastes implementation
def _e_step(self) -> None:
    """Vectorized implementation of e-step without worker loops."""
    # Compute diagonal contributions
    # shape: (n_task, n_workers, n_classes)
    diag_contrib = np.power(
        self.pi[np.newaxis, :, :],  # (1, n_workers, n_classes)
        self.crowd_matrix  # (n_task, n_workers, n_classes)
    )
    
    # Compute off-diagonal contributions
    # For each class j, we need to multiply pi_non_diag_values[k,j]^worker_labels[l] for all l != j
    mask = 1 - np.eye(self.n_classes)  # (n_casses, n_classes)
    
    # shape: (n_task, n_workers, n_classes, n_classes)
    off_diag_powers = np.power(
        self.pi_non_diag_values[np.newaxis, :, np.newaxis, :],  # (1, n_workers, 1, n_classes)
        self.crowd_matrix[:, :, :, np.newaxis] * mask[np.newaxis, np.newaxis, :, :]
    )
    
    off_diag_contrib = np.prod(off_diag_powers, axis=2)  # (n_task, n_workers, n_classes)
    
    worker_probs = diag_contrib * off_diag_contrib
    
    T = np.prod(worker_probs, axis=1) * self.rho[np.newaxis, :]  # (n_task, n_classes)
    
    self.denom_e_step = T.sum(axis=1, keepdims=True)
    self.T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)

In [15]:
dsd._e_step = MethodType(_e_step, dsd)
dsd.run(maxiter=100)
dsd.get_answers()

2025-03-25 10:17:12.107 | DEBUG    | __main__:init_T:103 - Size of T before calc: 1568
2025-03-25 10:17:12.108 | DEBUG    | __main__:init_T:107 - Size of T: 1568
Finished:  59%|█████▉    | 59/100 [00:00<00:00, 3013.22it/s]


array([2, 2, 8, 1, 4, 0, 5, 3, 7, 1, 0, 3, 1, 3, 4, 7, 4, 7, 3, 7])

In [21]:
# Bit faster, but not as memory efficient
def _e_step(self) -> None:
    T = np.zeros((self.n_task, self.n_classes))
    
    pi = self.pi.T  # (n_classes, n_workers)
    pi_non_diag = self.pi_non_diag_values.T  # (n_classes, n_workers)
    
    for j in range(self.n_classes):
        # Diagonal contributions for class j
        # shape: (n_task, n_workers)
        diag_contrib = np.power(pi[j], self.crowd_matrix[:, :, j])
        
        # Off-diagonal contributions for class j
        # Create mask for other classes
        other_classes = np.arange(self.n_classes) != j
        # shape: (n_task, n_workers, n_classes-1)
        off_diag_contrib = self.crowd_matrix[:, :, other_classes]
        # shape: (n_workers, n_classes-1)
        other_probs = pi_non_diag[j][:, np.newaxis]  # Broadcast to non-j classes
        
        # shape: (n_task, n_workers)
        off_diag_contrib = np.prod(
            np.power(other_probs, off_diag_contrib),
            axis=2
        )
        
        # shape: (n_task,)
        class_probs = np.prod(diag_contrib * off_diag_contrib, axis=1) * self.rho[j]
        T[:, j] = class_probs
    
    self.denom_e_step = T.sum(axis=1, keepdims=True)
    self.T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)

In [22]:
dsd._e_step = MethodType(_e_step, dsd)
dsd.run(maxiter=100)
dsd.get_answers()

2025-03-25 10:17:24.761 | DEBUG    | __main__:init_T:103 - Size of T before calc: 1568
2025-03-25 10:17:24.762 | DEBUG    | __main__:init_T:107 - Size of T: 1568
Finished:  59%|█████▉    | 59/100 [00:00<00:00, 1747.63it/s]


array([2, 2, 8, 1, 4, 0, 5, 3, 7, 1, 0, 3, 1, 3, 4, 7, 4, 7, 3, 7])

In [ ]:
# Should be comperable to Tanguy's implementation
def _e_step(self) -> None:
    """Vectorized E-step keeping task/class loops, vectorizing worker computations."""
    T = np.zeros((self.n_task, self.n_classes))
    
    for i in range(self.n_task):
        for j in range(self.n_classes):
            # Get all worker labels for task i (n_workers, n_classes)
            worker_labels = self.crowd_matrix[i]  # shape (n_workers, n_classes)

            # Vectorized computation for all workers simultaneously
            # Diagonal contributions: pi[k,j]^worker_labels[k,j]
            diag_contrib = np.power(self.pi[:, j], worker_labels[:, j])  # shape (n_workers,)

            # Off-diagonal contributions: product over l≠j of pi_non_diag[k,j]^worker_labels[k,l]
            mask = np.ones(self.n_classes, dtype=bool)
            mask[j] = False  # exclude current class j
            off_diag_labels = worker_labels[:, mask]  # shape (n_workers, n_classes-1)

            off_diag_contrib = np.prod(
                np.power(self.pi_non_diag_values[:, j][:, np.newaxis], off_diag_labels),
                axis=1
            )  # shape (n_workers,)

            # Combine contributions
            worker_probs = diag_contrib * off_diag_contrib  # shape (n_workers,)
            T[i, j] = np.prod(worker_probs) * self.rho[j]

    self.denom_e_step = T.sum(1, keepdims=True)
    self.T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)

In [24]:
dsd._e_step = MethodType(_e_step, dsd)
dsd.run(maxiter=100)
dsd.get_answers()

2025-03-25 10:17:30.142 | DEBUG    | __main__:init_T:103 - Size of T before calc: 1568
2025-03-25 10:17:30.143 | DEBUG    | __main__:init_T:107 - Size of T: 1568
Finished:  59%|█████▉    | 59/100 [00:00<00:00, 219.66it/s]       


array([2, 2, 8, 1, 4, 0, 5, 3, 7, 1, 0, 3, 1, 3, 4, 7, 4, 7, 3, 7])